In [ ]:
import torch
import numpy as np
from torchvision import transforms, datasets, utils
import torch.nn as nn
import matplotlib.pyplot as plt
import splitfolders

data_transform = transforms.Compose([
        transforms.CenterCrop (100),
        transforms.RandomHorizontalFlip(),
        transforms.ToTensor(),
        transforms.Normalize(mean=[0.5, 0.5, 0.5],
                             std=[0.5, 0.5, 0.5])
    ])

#transforms.Normalize(mean=[0.485, 0.456, 0.406],std=[0.229, 0.224, 0.225]) old normalize

#C:\Users\André\Desktop\tree_ML\image_labeled_input
#C:\Users\André\Desktop\tree_ML\images_labeled

splitfolders.ratio(r"./image_labeled_input/", output=r"./images_labeled/", seed=1337, ratio=(.7, .3), group_prefix=None)
#tree_dataset = datasets.ImageFolder(root=r'C:\Users\André\Desktop\tree_ML\image_labeled',
#                                           transform=data_transform)

train_dataset = datasets.ImageFolder(root=r'./images_labeled/train/',
                                           transform=data_transform)

val_dataset = datasets.ImageFolder(root=r'./images_labeled/val/',
                                           transform=data_transform)

dataset_unlabeled = datasets.ImageFolder(root=r'./images/',
                                           transform=data_transform)

#train_size = int(0.7 * len(tree_dataset))
#test_size = len(tree_dataset) - train_size
#train_dataset, test_dataset = torch.utils.data.random_split(tree_dataset, [train_size, test_size])

train_loader = torch.utils.data.DataLoader(train_dataset,
                                             batch_size=32, shuffle=True,
                                             num_workers=4)

val_loader = torch.utils.data.DataLoader(val_dataset,
                                             batch_size=32, shuffle=False,
                                             num_workers=4)

#claseeee = tree_dataset.classes
train_classes = train_dataset.classes
val_classes = val_dataset.classes

In [ ]:
def valid_imshow_data(data):
    data = np.asarray(data)
    if data.ndim == 2:
        return True
    elif data.ndim == 3:
        if 3 <= data.shape[2] <= 4:
            return True
        else:
            print('The "data" has 3 dimensions but the last dimension '
                  'must have a length of 3 (RGB) or 4 (RGBA), not "{}".'
                  ''.format(data.shape[2]))
            return False
    else:
        print('To visualize an image the data must be 2 dimensional or '
              '3 dimensional, not "{}".'
              ''.format(data.ndim))
        return False

In [ ]:
def imshow(inp, title=None):
    """Imshow for Tensor."""
    inp = inp.permute(1,2,0)
    plt.imshow(inp)
    if title is not None:
        plt.title(title)
    plt.pause(0.001)  # pause a bit so that plots are updated

In [ ]:
images, labels = next(iter(train_loader))

#print(im)
#print(im.shape)
#valid_imshow_data(im)
# change order of dimensions to fit imshow
#plt.imshow(im, norm=None)

out = utils.make_grid(images)

imshow(out, title=[train_classes[x] for x in labels])

print(len(train_dataset))
#print(dir(tree_dataset))

In [ ]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print(device)

In [ ]:
class ConvNet(nn.Module):
    def __init__(self):
        super(ConvNet, self).__init__()
        self.layer1 = nn.Sequential(
            nn.Conv2d(3, 32, kernel_size=5, stride=1, padding=2),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2, stride=2))
        self.layer2 = nn.Sequential(
            nn.Conv2d(32, 64, kernel_size=5, stride=1, padding=2),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2, stride=2))
        self.drop_out = nn.Dropout()
        self.fc1 = nn.Linear(25 * 25 * 64, 1000)
        self.fc2 = nn.Linear(1000, 10)
    
    def forward(self, x):
        out = self.layer1(x)
        out = self.layer2(out)
        out = out.reshape(out.size(0), -1)
        out = self.drop_out(out)
        out = self.fc1(out)
        out = self.fc2(out)
        return out

In [ ]:
# Hyperparameters
num_epochs = 5
num_classes = len(train_classes)
batch_size = 32
learning_rate = 0.001

In [ ]:
model = ConvNet()
# Loss and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

if torch.cuda.is_available():
    model = model.to(device)
    criterion = criterion.to(device)


# Train the model
model.train()
total_step = len(train_loader)
loss_list = []
acc_list = []
for epoch in range(num_epochs):
    for i, (images, labels) in enumerate(train_loader):
        
        images = images.to(device)
        labels = labels.to(device)
        
        # Run the forward pass
        outputs = model(images)
        loss = criterion(outputs, labels)
        loss_list.append(loss.item())

        # Backprop and perform Adam optimisation
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        # Track the accuracy
        total = labels.size(0)
        _, predicted = torch.max(outputs.data, 1)
        correct = (predicted == labels).sum().item()
        acc_list.append(correct / total)
        print("fuck", i, epoch)
        #if (i + 1) % 100 == 0:
        print('Epoch [{}/{}], Step [{}/{}], Loss: {:.4f}, Accuracy: {:.2f}%'
                  .format(epoch + 1, num_epochs, i + 1, total_step, loss.item(),
                          (correct / total) * 100))

In [ ]:
# Test the model
model.eval()
with torch.no_grad():
    correct = 0
    total = 0
    for images, labels in val_loader:
        images = images.to(device)
        labels = labels.to(device)
        
        outputs = model(images)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

    print('Test Accuracy of the model on the {} test images: {} %'.format(len(val_dataset), (correct / total) * 100))

# Save the model and plot
torch.save(model.state_dict(), './model/conv_net_model.ckpt')